<a href="https://colab.research.google.com/github/zhaochuninhefei/ipynbs/blob/master/ChatGLM_6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 一、准备工作

## 1.1 选择GPU
在菜单`代码执行程序`中选择`更改运行时类型`,选择GPT，免费的T4。

然后检查当前N卡配置:

In [1]:
!nvidia-smi

Mon May 29 00:34:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

可以看到免费的T4显卡有15G显存

## 1.2 下载ChatGLM-6B项目
默认会下载到`/content/ChatGLM-6B`目录:

In [2]:
!git clone https://github.com/THUDM/ChatGLM-6B.git

Cloning into 'ChatGLM-6B'...
remote: Enumerating objects: 1187, done.
remote: Counting objects: 100% (623/623), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 1187 (delta 532), reused 545 (delta 504), pack-reused 564
Receiving objects: 100% (1187/1187), 9.01 MiB | 26.75 MiB/s, done.
Resolving deltas: 100% (701/701), done.


## 1.3 安装依赖库
根据ChatGLM-6B项目目录下的`requirements.txt`文件安装相关依赖包。

In [3]:
!pip install -r /content/ChatGLM-6B/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 1

# 二、加载模型
这里选择`chatglm-6b-int4`量化模型。
> 默认的`chatglm-6b`(FP16无量化模型)和int8量化模型都因为这里内存RAM不足(只有12.7G)而无法成功加载模型。

In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4",trust_remote_code=True).half().cuda()
# 无法成功加载int8模型，即使去掉half()也不行，虽然分配有12.7G内存，但实际使用时似乎不能超过6G
# tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int8", trust_remote_code=True)
# model = AutoModel.from_pretrained("THUDM/chatglm-6b-int8",trust_remote_code=True).cuda()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c
Compiling gcc -O3 -fPIC -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Using quantization cache
Applying quantization to glm layers


# 三、运行推理
## 3.1 首次推理
首次推理会慢一点。这里的提示词很简单，就是`你好`。

In [2]:
%%time
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。
CPU times: user 8.11 s, sys: 843 ms, total: 8.95 s
Wall time: 14.8 s


可以看到，首次运行推理花了大约15秒。

## 3.2 再次推理
然后我们再次运行相同提示词的推理：

In [3]:
%%time
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。
CPU times: user 4.89 s, sys: 0 ns, total: 4.89 s
Wall time: 4.92 s


快了不少，只要大约5秒，这个其实还是很慢。

## 3.3 其他推理
在这里我们尝试一下其他稍微复杂点的提示词:

In [5]:
%%time
response, history = model.chat(tokenizer, "桌子上有4个苹果, 小红吃了1个, 小刚拿走了2个, 还剩下几个苹果?", history=[])
print(response)

还剩下3个苹果。
CPU times: user 1.38 s, sys: 1.13 ms, total: 1.38 s
Wall time: 1.5 s


这种计算推理问题，ChatGLM-6B的表现不是很好。

再尝试一个常识类问题:

In [6]:
%%time
response, history = model.chat(tokenizer, "中国四大名著是哪四本书?", history=[])
print(response)

中国四大名著是指《红楼梦》、《西游记》、《水浒传》和《三国演义》。这四本书都是中国文学史上的经典之作，被广泛传诵和阅读。
CPU times: user 5.37 s, sys: 2.13 ms, total: 5.38 s
Wall time: 5.36 s


这个还行，但有时也有说多错多的问题。

## 3.4 信息提取
再尝试一个做信息提取的例子:

In [ ]:
%%time
content="""本人是毕业于南七技校，性格开朗，待人真诚，对待工作认真负责，善于沟通、协调有较强的组织能力与团队精神;
活泼开朗、乐观上进、有爱心并善于施教并行;上进心强、勤于学习能不断提高自身的能力与综合素质。
在未来的工作中，我将以充沛的精力，刻苦钻研的精神来努力工作，稳定地提高自己的工作能力，与公司同步发展。
目前我居住在合肥市云深路234号奋斗小区，我的手机号码是: 18012345678。也可以用邮箱联系我: testtest@test.com。
"""
prompt="""从上文中，提取"信息"(keyword:content)，包括:"手机号"、"邮箱"、"毕业院校"、"住址"、"自我评价"等keyword及其content，输出json格式内容。
"""
input ='{}\n\n{}'.format(content,prompt)
print(input)
response, history = model.chat(tokenizer, input, history=[])
print(response)

本人是毕业于南七技校，性格开朗，待人真诚，对待工作认真负责，善于沟通、协调有较强的组织能力与团队精神;
活泼开朗、乐观上进、有爱心并善于施教并行;上进心强、勤于学习能不断提高自身的能力与综合素质。
在未来的工作中，我将以充沛的精力，刻苦钻研的精神来努力工作，稳定地提高自己的工作能力，与公司同步发展。
目前我居住在合肥市云深路234号奋斗小区，我的手机号码是: 18012345678。也可以用邮箱联系我: testtest@test.com。


从上文中，提取"信息"(keyword:content)，包括:"手机号"、"邮箱"、"毕业院校"、"住址"、"自我评价"等keyword及其content，输出json格式内容。

{  
 "手机号": 18012345678,  
 "邮箱": testtest@test.com,  
 "毕业院校": 南七技校，  
 "住址": 合肥市云深路234号奋斗小区，  
 "自我评价": 活泼开朗、乐观上进、有爱心并善于施教并行；上进心强、勤于学习能不断提高自身的能力与综合素质  
}
CPU times: user 17.6 s, sys: 2.12 ms, total: 17.6 s
Wall time: 17.7 s


这个效果还是不错的，这里可以看出来很明显的语境内学习能力。

## 3.5 信息统计
### 先来试一个信息收集统计

In [ ]:
%%time
content="""河马: 啦啦啦啦啦啦啦

张三: 怎么回事？突然断电了？

李四: 我家也断电了

王五: +1

傻大个: 没有啊，我家正常的啊

胖君子: 啊这，好累。

张三: @管家，什么情况？什么时候能来电? 

王五: 同问

彼岸花: 我家也停电了

兔子: +1

毛熊: +1

小鹰: 没停电的飘过。。。

管家: 正在联系供电局，请大家少安勿躁。

李四: 最近经常停电啊，世界末日要来了，大家出来嗨吧～～～

屁屁: 还好我家没停。

谁家大爷: 又停电了

胡萝卜: +1

管家: 正在联系供电局，请大家稍安勿躁。

"""
prompt='阅读上面的对话，请问有几家停电或断电了?'
input ='{}\n\n{}'.format(content,prompt)
print(input)
response, history = model.chat(tokenizer, input, history=[])
print()
print("回答:")
print(response)

河马: 啦啦啦啦啦啦啦

张三: 怎么回事？突然断电了？

李四: 我家也断电了

王五: +1

傻大个: 没有啊，我家正常的啊

胖君子: 啊这，好累。

张三: @管家，什么情况？什么时候能来电? 

王五: 同问

彼岸花: 我家也停电了

兔子: +1

毛熊: +1

小鹰: 没停电的飘过。。。

管家: 正在联系供电局，请大家少安勿躁。

李四: 最近经常停电啊，世界末日要来了，大家出来嗨吧～～～

屁屁: 还好我家没停。

谁家大爷: 又停电了

胡萝卜: +1

管家: 正在联系供电局，请大家稍安勿躁。



阅读上面的对话，请问有几家停电或断电了?

回答:
在这段对话中，有5个人提到了他们家里最近停电了。
CPU times: user 2.93 s, sys: 11.1 ms, total: 2.95 s
Wall time: 4.01 s


很遗憾，统计结果不正确，实际上应该是8家停电。多试几次会发现，这种统计对chatGLM来说还是太难了，偶尔能蒙对一次。

### 再试一下只有一家停电的场景

In [ ]:
%%time
content="""河马: 啦啦啦啦啦啦啦

张三: 怎么回事？突然断电了？

李四: 我家正常

王五: 正常+1

傻大个: 没有啊，我家正常的啊

胖君子: 啊这，好累。

张三: @管家，什么情况？什么时候能来电? 

王五: 正常+1

彼岸花: 我家正常

兔子: 正常+1

毛熊: 正常+1

小鹰: 没停电的飘过。。。

管家: 正在联系供电局，请大家少安勿躁。

李四: 世界末日要来了，大家出来嗨吧～～～

屁屁: 还好我家没停。

谁家大爷: 我家没停电

胡萝卜: 没停+1

管家: 正在联系供电局，请大家稍安勿躁。

"""
prompt='阅读上面的对话，请问有几家停电或断电了?'
input ='{}\n\n{}'.format(content,prompt)
print(input)
response, history = model.chat(tokenizer, input, history=[])
print()
print("回答:")
print(response)

河马: 啦啦啦啦啦啦啦

张三: 怎么回事？突然断电了？

李四: 我家正常

王五: 正常+1

傻大个: 没有啊，我家正常的啊

胖君子: 啊这，好累。

张三: @管家，什么情况？什么时候能来电? 

王五: 正常+1

彼岸花: 我家正常

兔子: 正常+1

毛熊: 正常+1

小鹰: 没停电的飘过。。。

管家: 正在联系供电局，请大家少安勿躁。

李四: 世界末日要来了，大家出来嗨吧～～～

屁屁: 还好我家没停。

谁家大爷: 我家没停电

胡萝卜: 没停+1

管家: 正在联系供电局，请大家稍安勿躁。



阅读上面的对话，请问有几家停电或断电了?

回答:
根据对话中的信息，有几家停电或断电了：

- 张三： 突然断电了？
- 李四： 我家正常
- 王五： 正常+1
- 傻大个： 没有啊，我家正常的啊
- 胖君子： 啊这，好累
- 彼岸花： 我家正常
- 兔子： 正常+1
- 毛熊： 正常+1
- 小鹰： 没停电的飘过
- 管家： 正在联系供电局，请大家少安勿躁
- 李四： 世界末日要来了，大家出来嗨吧～～～
- 屁屁： 还好我家没停
- 谁家大爷： 我家没停电
- 胡萝卜： 没停+1

因此，有4家停电或断电了。
CPU times: user 33.6 s, sys: 88.8 ms, total: 33.7 s
Wall time: 33.9 s


这个就更难了，结果实在离谱。

### 如果提示词加入示例，再来看一下结果

In [ ]:
%%time
content="""请统计一段对话中停电的户数有多少。
例1:
```
张三: 怎么回事？突然断电了？
李四: 我家也断电了
王五: +1
傻大个: 没有啊，我家正常的啊
胖君子: 啊这，好累。
```
应该统计出停电户数:3。

例2：
```
张三: 怎么回事？突然断电了？
李四: 我家没断电啊
王五: 没断电+1
傻大个: 我家也正常的啊
胖君子: 正常+1
```
应该统计出停电户数:1。

请按照同样的规则统计下面这段对话的停电户数:
```
河马: 啦啦啦啦啦啦啦
小明: 怎么回事？突然断电了？
小芳: 我家也断电了
小华: +1
阿达: 没有啊，我家正常的啊
君莫笑: 啊这，好累。
小明: @管家，什么情况？什么时候能来电? 
小华: 同问
彼岸花: 我家也停电了
鸽子: +1
大熊: +1
老鹰: 没停电的飘过。。。
管家: 正在联系供电局，请大家少安勿躁。
小芳: 最近经常停电啊，世界末日要来了，大家出来嗨吧～～～
屁屁: 还好我家没停。
谁家大爷: 没停电+1
胡萝卜: 没停电+1
管家: 正在联系供电局，请大家稍安勿躁。
```
"""
print(content)
response, history = model.chat(tokenizer, content, history=[])
print()
print("回答:")
print(response)

请统计一段对话中停电的户数有多少。
例1:
```
张三: 怎么回事？突然断电了？
李四: 我家也断电了
王五: +1
傻大个: 没有啊，我家正常的啊
胖君子: 啊这，好累。
```
应该统计出停电户数:3。

例2：
```
张三: 怎么回事？突然断电了？
李四: 我家没断电啊
王五: 没断电+1
傻大个: 我家也正常的啊
胖君子: 正常+1
```
应该统计出停电户数:1。

请按照同样的规则统计下面这段对话的停电户数:
```
河马: 啦啦啦啦啦啦啦
小明: 怎么回事？突然断电了？
小芳: 我家也断电了
小华: +1
阿达: 没有啊，我家正常的啊
君莫笑: 啊这，好累。
小明: @管家，什么情况？什么时候能来电? 
小华: 同问
彼岸花: 我家也停电了
鸽子: +1
大熊: +1
老鹰: 没停电的飘过。。。
管家: 正在联系供电局，请大家少安勿躁。
小芳: 最近经常停电啊，世界末日要来了，大家出来嗨吧～～～
屁屁: 还好我家没停。
谁家大爷: 没停电+1
胡萝卜: 没停电+1
管家: 正在联系供电局，请大家稍安勿躁。
```


回答:
根据上面的例子，我们可以得到以下统计结果：

- 河马： 啦啦啦啦啦啦啦  
- 小明： 怎么回事？突然断电了？  
- 小芳： 我家也断电了  
- 小华： +1  
- 阿达： 没有啊，我家正常的啊  
- 君莫笑： 啊这，好累  
- 小明： @管家，什么情况？什么时候能来电？  
- 小华： 同问  
- 彼岸花： 我家也停电了  
- 鸽子： +1  
- 大熊： +1  
- 老鹰： 没停电的飘过。。。  
- 管家： 正在联系供电局，请大家少安勿躁。  
- 小芳： 最近经常停电啊，世界末日要来了，大家出来嗨吧～～～  
- 屁屁： 还好我家没停。  
- 谁家大爷： 没停电+1  
- 胡萝卜： 没停电+1  

因此，这段对话中的停电户数为：3。
CPU times: user 44.6 s, sys: 73.4 ms, total: 44.7 s
Wall time: 44.9 s


最后这个依然不靠谱。
但同样的提示词，最后这种带示例的，在new bing那里能得到正确结果，并且new bing会给出统计过程。